In [1]:
import copy
import random
import os
import time

from IPython.display import clear_output, display, HTML

from collections import Counter
from PIL import Image

import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
sns.set(style="darkgrid")

import tensorflow as tf

os.environ["CUDA_VISIBLE_DEVICES"] = ','.join(
    str(i) for i in range(len(tf.config.experimental.list_physical_devices('GPU'))))
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"
for i in range(len(tf.config.experimental.list_physical_devices('GPU'))):
    tf.config.experimental.set_memory_growth(tf.config.experimental.list_physical_devices('GPU')[i], True)

keras = tf.keras
# from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPool2D
from keras.utils import array_to_img, img_to_array, load_img

from sklearn.model_selection import train_test_split

# https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/DBW86T

2022-11-23 07:48:18.742186: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-23 07:48:19.074951: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-23 07:48:19.836155: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-23 07:48:19.836230: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
# Constants
BASE_PATH = "../src/data"
DATA_PATH = f"{BASE_PATH}/external"
STATE_PATH = f"{BASE_PATH}/state"
IMAGE_PATH = f'{DATA_PATH}/ISIC_2019_Training_Input'
IMAGE_TYPE = 'jpg'
MODELS_PATH = f"{BASE_PATH}/models"
TARGET_SIZE = (224, 224)
CHANNELS = 3
CONTRAST_FACTOR = 3  # 2 is performing better and so choosing that instead
# CONTRAST_FACTOR = 2
DELTA = 0.3
IS_FLIP_ON = True
RANDOM_STATE = 42
# KERNEL_SIZE = (5, 5)  # decreasing seeing 3,3 doing better
KERNEL_SIZE = (3, 3)
# LEARNING_RATE = 0.001  # decreasing seeing too much ripple effect
LEARNING_RATE = 0.0005
BATCH_SIZE = 32
TEST_SIZE = 0.2
# EPOCHS = 5  # validation accuracy seems to increase with the new learning rate and so increasing the number of epochs
EPOCHS = 25

In [3]:
def reset_random_seeds(seed=RANDOM_STATE):
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

def clear_session():
    tf.keras.backend.clear_session()

In [4]:
df_metadata = pd.read_csv(f'{DATA_PATH}/ISIC_2019_Training_Metadata.csv')
df_truth = pd.read_csv(f'{DATA_PATH}/ISIC_2019_Training_GroundTruth.csv')

In [5]:
labels = df_truth.columns
labels = labels[1:]
print(labels)
label_mapping = {i: label for i, label in enumerate(labels)}
print(label_mapping)

Index(['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK'], dtype='object')
{0: 'MEL', 1: 'NV', 2: 'BCC', 3: 'AK', 4: 'BKL', 5: 'DF', 6: 'VASC', 7: 'SCC', 8: 'UNK'}


In [6]:
dense_labels = df_truth[labels]
dense_labels = dense_labels.values
print('Dense Labels...')
display(dense_labels)
truth_labels = np.argmax(dense_labels, axis=-1)
print('Truth Labels...')
display(truth_labels)

Dense Labels...


array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Truth Labels...


array([1, 1, 0, ..., 0, 1, 4])

In [7]:
df_truth['label'] = truth_labels.tolist()
display(df_truth)

,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK,label
0,ISIC_0000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,ISIC_0000001,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,ISIC_0000002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,ISIC_0000003,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,ISIC_0000004,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
25326,ISIC_0073247,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2
25327,ISIC_0073248,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4
25328,ISIC_0073249,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
25329,ISIC_0073251,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [8]:
df_joined = df_metadata.join(df_truth.set_index('image'), on='image', how='left')
display(df_joined)

,image,age_approx,anatom_site_general,lesion_id,sex,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK,label
0,ISIC_0000000,55.0,anterior torso,NaN,female,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,ISIC_0000001,30.0,anterior torso,NaN,female,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,ISIC_0000002,60.0,upper extremity,NaN,female,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,ISIC_0000003,30.0,upper extremity,NaN,male,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,ISIC_0000004,80.0,posterior torso,NaN,male,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25326,ISIC_0073247,85.0,head/neck,BCN_0003925,female,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2
25327,ISIC_0073248,65.0,anterior torso,BCN_0001819,male,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4
25328,ISIC_0073249,70.0,lower extremity,BCN_0001085,male,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
25329,ISIC_0073251,55.0,palms/soles,BCN_0002083,female,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [9]:
# Site details add more information to model and so fill with unknown if not available
print(f"Count of null values in anatom_site_general column before null fill: {sum(df_joined['anatom_site_general'].isnull())}")
df_joined['anatom_site_general'] = df_joined['anatom_site_general'].fillna('unknown')
print(f"Count of null values in anatom_site_general column after null fill: {sum(df_joined['anatom_site_general'].isnull())}")

Count of null values in anatom_site_general column before null fill: 2631
Count of null values in anatom_site_general column after null fill: 0


In [10]:
# Sex details add more information to model and so fill with unknown if not available
print(f"Count of null values in sex column before null fill: {sum(df_joined['sex'].isnull())}")
df_joined['sex'] = df_joined['sex'].fillna('unknown')
print(f"Count of null values in sex column after null fill: {sum(df_joined['sex'].isnull())}")

Count of null values in sex column before null fill: 384
Count of null values in sex column after null fill: 0


In [11]:
# Age is important for the model and so we remove rows with na/inf/zero values
df_cleaned = df_joined[~df_joined['age_approx'].isnull()]
print(f"Observations in cleaned dataframe before na/inf/zero remove: {len(df_cleaned)}")
df_cleaned.replace([np.inf, -np.inf], np.nan, inplace=True)
df_cleaned.dropna(how="all", inplace=True)
df_cleaned = df_cleaned[df_cleaned['age_approx'] > 0]
print(f"Observations in cleaned dataframe after na/inf/zero remove: {len(df_cleaned)}")

Observations in cleaned dataframe before na/inf/zero remove: 24894
Observations in cleaned dataframe after na/inf/zero remove: 24840


In [12]:
# Convert age column to uint8 for resampling
# df_cleaned['age_approx'] = df_cleaned['age_approx'].astype(np.uint8)
# display(df_cleaned)

In [13]:
# decreasing the scope of anatomy site can provide more information and so getting a more generic information
df_cleaned['anatomy_site'] = df_cleaned['anatom_site_general'].apply(lambda anatomy: anatomy.split()[-1])
display(df_cleaned)

,image,age_approx,anatom_site_general,lesion_id,sex,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK,label,anatomy_site
0,ISIC_0000000,55.0,anterior torso,NaN,female,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,torso
1,ISIC_0000001,30.0,anterior torso,NaN,female,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,torso
2,ISIC_0000002,60.0,upper extremity,NaN,female,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,extremity
3,ISIC_0000003,30.0,upper extremity,NaN,male,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,extremity
4,ISIC_0000004,80.0,posterior torso,NaN,male,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,torso
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25326,ISIC_0073247,85.0,head/neck,BCN_0003925,female,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2,head/neck
25327,ISIC_0073248,65.0,anterior torso,BCN_0001819,male,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4,torso
25328,ISIC_0073249,70.0,lower extremity,BCN_0001085,male,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,extremity
25329,ISIC_0073251,55.0,palms/soles,BCN_0002083,female,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,palms/soles


In [14]:
with open(f"{STATE_PATH}/images_seen.txt") as seen_file:
    images_seen = seen_file.read()
    images_seen = images_seen.split('\n')
    images_seen = set(images_seen)
df_cleaned['is_seen'] = df_cleaned['image'].apply(lambda name: name in images_seen)

df_seen = df_cleaned[df_cleaned['is_seen']].copy()
display(df_seen)
df_test = df_cleaned[df_cleaned['is_seen'] == False].copy()
# Limit the number of test results to speed up experiments
df_test = df_test.head(df_seen.shape[0])
display(df_test)

,image,age_approx,anatom_site_general,lesion_id,sex,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK,label,anatomy_site,is_seen
2915,ISIC_0024318,65.0,lower extremity,HAM_0002450,female,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5,extremity,True
2967,ISIC_0024370,55.0,unknown,HAM_0001780,male,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,unknown,True
2983,ISIC_0024386,40.0,lower extremity,HAM_0005112,female,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5,extremity,True
3072,ISIC_0024475,35.0,head/neck,HAM_0003873,male,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,head/neck,True
3114,ISIC_0024517,65.0,posterior torso,HAM_0001894,male,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7,torso,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25256,ISIC_0073141,45.0,anterior torso,BCN_0005520,female,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5,torso,True
25259,ISIC_0073144,75.0,anterior torso,BCN_0004091,female,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,torso,True
25287,ISIC_0073193,35.0,anterior torso,BCN_0002147,male,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5,torso,True
25289,ISIC_0073195,70.0,lower extremity,BCN_0005492,male,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7,extremity,True


,image,age_approx,anatom_site_general,lesion_id,sex,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK,label,anatomy_site,is_seen
0,ISIC_0000000,55.0,anterior torso,NaN,female,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,torso,False
1,ISIC_0000001,30.0,anterior torso,NaN,female,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,torso,False
2,ISIC_0000002,60.0,upper extremity,NaN,female,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,extremity,False
3,ISIC_0000003,30.0,upper extremity,NaN,male,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,extremity,False
4,ISIC_0000004,80.0,posterior torso,NaN,male,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,torso,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1745,ISIC_0013235_downsampled,45.0,posterior torso,MSK4_0011691,male,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,torso,False
1746,ISIC_0013236_downsampled,75.0,upper extremity,MSK4_0011692,male,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4,extremity,False
1747,ISIC_0013238_downsampled,75.0,upper extremity,MSK4_0010809,female,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,extremity,False
1748,ISIC_0013242_downsampled,85.0,upper extremity,MSK4_0010725,male,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,extremity,False


## Baseline

Predicting everything as MEL (label = 0) since that is the most dangerous type of skin lesion!!!

In [15]:
accuracy_test = (df_test['label'] == 0).sum() / df_test.shape[0]
print(f"Accuracy on test data: {accuracy_test * 100:.2f}%")

Accuracy on test data: 14.87%


In [16]:
def load_img_data(img_names):
    images = []
    for img_name in img_names:
        img = load_img(f"{IMAGE_PATH}/{img_name}.{IMAGE_TYPE}", target_size=TARGET_SIZE)
        img = img_to_array(img)
        images.append(img)
    return np.stack(images)

In [17]:
def aug_img_data(images, size=TARGET_SIZE, delta=DELTA, contrast_factor=CONTRAST_FACTOR, is_flip_on=IS_FLIP_ON, **kwargs):
    # make a copy of images
    images_aug = copy.deepcopy(images)
    # image resize
    images_aug = tf.image.resize(images_aug, size=size)

    # brightness
    images_aug = tf.image.adjust_brightness(images_aug, delta=delta)
    # contrast
    images_aug = tf.image.adjust_contrast(images_aug, contrast_factor=contrast_factor)
    # random flip
    if is_flip_on:
        images_aug = tf.image.random_flip_left_right(images_aug)

    return images_aug

In [18]:
img_seen = load_img_data(df_seen['image'])
y_seen = df_seen['label']

img_train, img_val, y_train, y_val = train_test_split(img_seen, y_seen, test_size=TEST_SIZE, random_state=RANDOM_STATE)
print(f"Length of training data: {img_train.shape[0]}")
print(f"Length of validation data: {img_val.shape[0]}")

img_test = load_img_data(df_test['image'])
y_test = df_test['label']
print(f"Length of test data: {img_test.shape[0]}")

Length of training data: 1216
Length of validation data: 304
Length of test data: 1520


In [19]:
def plot_history(hist):
    x_arr = np.arange(len(hist['loss'])) + 1

    fig = plt.figure(figsize=(12, 4))
    ax = fig.add_subplot(1, 2, 1)
    ax.plot(x_arr, hist['loss'], '-o', label='Train loss')
    ax.plot(x_arr, hist['val_loss'], '--<', label='Validation loss')
    ax.legend(fontsize=15)
    ax.set_xlabel('Epoch', size=15)
    ax.set_ylabel('Loss', size=15)

    ax = fig.add_subplot(1, 2, 2)
    ax.plot(x_arr, hist['accuracy'], '-o', label='Train acc.')
    ax.plot(x_arr, hist['val_accuracy'], '--<', label='Validation acc.')
    ax.legend(fontsize=15)
    ax.set_xlabel('Epoch', size=15)
    ax.set_ylabel('Accuracy', size=15)
    plt.show()

## CNN

In [20]:
def build_cnn_model(kernel_size=KERNEL_SIZE, strides=(1, 1), pool_size=(2, 2), optimizer='Adam', learning_rate=LEARNING_RATE, conv_filters=[32, 64], hidden_units=[1024, 256], dropout_rate=0.8, **kwargs):
# def build_cnn_model(kernel_size=(5, 5), strides=(1, 1), pool_size=(2, 2), optimizer='Adam', learning_rate=0.001, conv_filters=[32, 64], hidden_units=[1024], dropout_rate=0.5, **kwargs):
    clear_session()
    reset_random_seeds()

    model = keras.Sequential()

    for i, filters in enumerate(conv_filters):
        i += 1
        model.add(keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding='same', name=f'conv_{i}', activation='relu'))
        model.add(keras.layers.MaxPool2D(pool_size=pool_size, name=f'pool_{i}'))

    model.add(keras.layers.Flatten())

    for i, units in enumerate(hidden_units):
        i += 1
        model.add(keras.layers.Dropout(rate=dropout_rate))
        model.add(keras.layers.Dense(units=units, name=f'fc_{i}', activation='relu'))

    model.add(keras.layers.Dropout(rate=dropout_rate))
    model.add(keras.layers.Dense(units=len(labels), name='output', activation="softmax"))
    model.build(input_shape=(None, *TARGET_SIZE, 3))

    optimizer_mapping = {
        'sgd': keras.optimizers.SGD,
        'adam': keras.optimizers.Adam,
    }
    optimizer = optimizer_mapping[optimizer.lower()]
    optimizer = optimizer(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=['accuracy'])
    return model

In [21]:
def build_embedding_df(model, data_img, index):
    # get the flatten, penultimate, and output layers
    # layer_outputs = [model.get_layer('flatten'), model.layers[-3], model.layers[-1]]
    # layer_outputs = [layer.output for layer in layer_outputs]
    # model_img_embedding = keras.models.Model(inputs=model_cnn.input, outputs=layer_outputs)

    # get penultimate layer
    model_img_embedding = keras.models.Model(inputs=model_cnn.input, outputs=model.layers[-3].output)
    embedding_img = model_img_embedding.predict(data_img)
    print(embedding_img.shape)

    embedding_cols = [f"emb_{i+1}" for i in range(embedding_img.shape[1])]
    embedding_df = pd.DataFrame(embedding_img, columns=embedding_cols, index=index)
    embedding_df.reset_index(inplace=True)
    return embedding_df.rename(columns={'index': 'image'})

# augment train data
img_seen_aug = aug_img_data(img_seen)
img_seen_aug = tf.concat([img_seen, img_seen_aug], axis=0)
y_seen_aug = tf.concat([y_seen, y_seen], axis=0)

# shuffle the data
reset_random_seeds()
shuffle = tf.random.shuffle(tf.range(tf.shape(img_seen_aug)[0], dtype=tf.int32))
img_seen_aug = tf.gather(img_seen_aug, shuffle)
y_seen_aug = tf.gather(y_seen_aug, shuffle).numpy()
print(f"Length of augmented seen data: {img_seen_aug.shape[0]}")

if 'model_cnn' in locals() or 'model_cnn' in globals():
    del model_cnn
model_cnn = build_cnn_model()
model_cnn.summary()

history_cnn = model_cnn.fit(img_seen_aug, y_seen_aug, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=TEST_SIZE).history
plot_history(history_cnn)

i = 1100
print(f"embeddings: {build_embedding_df(model_cnn, img_seen_aug[i:i + 1]).to_numpy()}")
print(f"predicts: {model_cnn.predict(img_seen_aug[i:i+1])}")
print(f"ground-truth: {y_seen_aug[i:i+1]}")

default_args = {
    "kernel_size": KERNEL_SIZE,
    "strides": (1, 1),
    "pool_size": (2, 2),
    "optimizer": "Adam",
    "learning_rate": LEARNING_RATE,
    "delta": DELTA,
    "contrast_factor": CONTRAST_FACTOR,
    "is_flip_on": IS_FLIP_ON
}
new_args = {
    # "kernel_size": (3, 3),
    "strides": (2, 2),
    # "pool_size": (3, 3),
    # "optimizer": "SGD",  # unable to train and so not considering this
    # "learning_rate": 0.01,  # this is causing ripple effect
    "delta": 0.1,  # this is performing better and so using as default
    "contrast_factor": 2,  # this is performing better and so using as default
    "is_flip_on": False
}
# history_cnn = history_cnn.history
results = [[
    history_cnn['accuracy'][-1],
    history_cnn['val_accuracy'][-1],
    default_args['kernel_size'],
    default_args['strides'],
    default_args['pool_size'],
    default_args['learning_rate'],
    default_args['optimizer'],
    default_args['delta'],
    default_args['contrast_factor'],
    default_args['is_flip_on']
]]

for k, v in new_args.items():
    time.sleep(30)
    # clear_output(wait=True)
    exp_args = copy.deepcopy(default_args)
    exp_args[k] = v
    print(f"Experiment: {exp_args}")
    print(f"Modified: {k} = {v}")

    img_seen_aug = aug_img_data(img_seen, **exp_args)
    img_seen_aug = tf.concat([img_seen, img_seen_aug], axis=0)
    y_seen_aug = tf.concat([y_seen, y_seen], axis=0)

    # shuffle the data
    reset_random_seeds()
    shuffle = tf.random.shuffle(tf.range(tf.shape(img_seen_aug)[0], dtype=tf.int32))
    img_seen_aug = tf.gather(img_seen_aug, shuffle)
    y_seen_aug = tf.gather(y_seen_aug, shuffle).numpy()
    # print(f"Length of augmented seen data: {img_seen_aug.shape[0]}")

    if 'model_cnn' in locals() or 'model_cnn' in globals():
        del model_cnn
    model_cnn = build_cnn_model(**exp_args)
    model_cnn.summary()

    history_cnn = model_cnn.fit(img_seen_aug, y_seen_aug, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=TEST_SIZE).history
    plot_history(history_cnn)

    results.append([
        history_cnn['accuracy'][-1],
        history_cnn['val_accuracy'][-1],
        exp_args['kernel_size'],
        exp_args['strides'],
        exp_args['pool_size'],
        exp_args['learning_rate'],
        exp_args['optimizer'],
        exp_args['delta'],
        exp_args['contrast_factor'],
        exp_args['is_flip_on']
    ])

    i = 1100
    print(f"embeddings: {build_embedding_df(model_cnn, img_seen_aug[i:i + 1]).to_numpy()}")
    print(f"predicts: {model_cnn.predict(img_seen_aug[i:i+1])}")
    print(f"ground-truth: {y_seen_aug[i:i+1]}")

df_cnn_results = pd.DataFrame(results)
display(HTML(df_cnn_results.to_html(index=False)))

## Create Image Embeddings

#### Selected CNN

In [22]:
CNN_MODEL_FILENAME = f"{MODELS_PATH}/model_cnn.h5"
if os.path.isfile(CNN_MODEL_FILENAME):
    model_cnn = keras.models.load_model(CNN_MODEL_FILENAME)
else:
    default_args = {
        "kernel_size": KERNEL_SIZE,
        "strides": (1, 1),
        "pool_size": (2, 2),
        "optimizer": "Adam",
        "learning_rate": LEARNING_RATE,
        "delta": DELTA,
        "contrast_factor": CONTRAST_FACTOR,
        "is_flip_on": IS_FLIP_ON
    }
    exp_args = {
        **default_args,
        "strides": (2, 2),
        "contrast_factor": 2
    }

    # time.sleep(30)
    # clear_output(wait=True)
    print(f"Experiment: {exp_args}")

    img_seen_aug = aug_img_data(img_seen, **exp_args)
    img_seen_aug = tf.concat([img_seen, img_seen_aug], axis=0)
    y_seen_aug = tf.concat([y_seen, y_seen], axis=0)

    # shuffle the data
    reset_random_seeds()
    shuffle = tf.random.shuffle(tf.range(tf.shape(img_seen_aug)[0], dtype=tf.int32))
    img_seen_aug = tf.gather(img_seen_aug, shuffle)
    y_seen_aug = tf.gather(y_seen_aug, shuffle).numpy()

    if 'model_cnn' in locals() or 'model_cnn' in globals():
        del model_cnn
    model_cnn = build_cnn_model(**exp_args)
    model_cnn.fit(img_seen_aug, y_seen_aug, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=TEST_SIZE)
    model_cnn.save(CNN_MODEL_FILENAME)

model_cnn.summary()

Experiment: {'kernel_size': (3, 3), 'strides': (2, 2), 'pool_size': (2, 2), 'optimizer': 'Adam', 'learning_rate': 0.0005, 'delta': 0.3, 'contrast_factor': 2, 'is_flip_on': True}


2022-11-23 07:48:37.036861: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-23 07:48:37.039356: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 915210240 exceeds 10% of free system memory.
2022-11-23 07:48:37.362249: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 915210240 exceeds 10% of free system memory.
2022-11-23 07:48:37.678726: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 915210240 exceeds 10% of free system memory.
2022-11-23 07:48:37.803028: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 915210240 exceeds 10% of free system memory.
2022-11-23 07:48:38.261643: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation o

Epoch 1/25
76/76 [==============================] - 10s 133ms/step - loss: 26.3794 - accuracy: 0.1086 - val_loss: 2.1661 - val_accuracy: 0.1513
Epoch 2/25
76/76 [==============================] - 10s 128ms/step - loss: 2.4068 - accuracy: 0.1361 - val_loss: 2.1692 - val_accuracy: 0.1464
Epoch 3/25
76/76 [==============================] - 9s 123ms/step - loss: 2.2929 - accuracy: 0.1303 - val_loss: 2.1618 - val_accuracy: 0.1497
Epoch 4/25
76/76 [==============================] - 9s 123ms/step - loss: 2.2368 - accuracy: 0.1246 - val_loss: 2.1542 - val_accuracy: 0.1414
Epoch 5/25
76/76 [==============================] - 9s 125ms/step - loss: 2.2023 - accuracy: 0.1423 - val_loss: 2.1464 - val_accuracy: 0.1628
Epoch 6/25
76/76 [==============================] - 9s 123ms/step - loss: 2.1886 - accuracy: 0.1336 - val_loss: 2.1398 - val_accuracy: 0.1645
Epoch 7/25
76/76 [==============================] - 10s 125ms/step - loss: 2.1568 - accuracy: 0.1312 - val_loss: 2.1312 - val_accuracy: 0.1562
Ep

In [23]:
df_embeddings_img_seen = build_embedding_df(model_cnn, img_seen, df_seen['image'].to_list())
display(df_embeddings_img_seen)

df_embeddings_img_test = build_embedding_df(model_cnn, img_test, df_test['image'].to_list())
display(df_embeddings_img_test)

48/48 [==============================] - 1s 23ms/step
(1520, 256)


,image,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,...,emb_247,emb_248,emb_249,emb_250,emb_251,emb_252,emb_253,emb_254,emb_255,emb_256
0,ISIC_0024318,0.0,0.139611,0.0,0.000000,0.043557,0.0,0.073905,0.175965,0.0,...,0.0,0.0,0.0,0.0,0.0,0.086808,0.0,0.0,0.033308,0.0
1,ISIC_0024370,0.0,0.139611,0.0,0.000000,0.043557,0.0,0.073905,0.175965,0.0,...,0.0,0.0,0.0,0.0,0.0,0.086808,0.0,0.0,0.033308,0.0
2,ISIC_0024386,0.0,0.139611,0.0,0.000000,0.043557,0.0,0.073905,0.175965,0.0,...,0.0,0.0,0.0,0.0,0.0,0.086808,0.0,0.0,0.033308,0.0
3,ISIC_0024475,0.0,0.139611,0.0,0.000000,0.043557,0.0,0.073905,0.175965,0.0,...,0.0,0.0,0.0,0.0,0.0,0.086808,0.0,0.0,0.033308,0.0
4,ISIC_0024517,0.0,0.139611,0.0,0.000000,0.043557,0.0,0.073905,0.175965,0.0,...,0.0,0.0,0.0,0.0,0.0,0.086808,0.0,0.0,0.033308,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1515,ISIC_0073141,0.0,0.139611,0.0,0.000000,0.043557,0.0,0.073905,0.175965,0.0,...,0.0,0.0,0.0,0.0,0.0,0.086808,0.0,0.0,0.033308,0.0
1516,ISIC_0073144,0.0,0.139611,0.0,0.000000,0.043557,0.0,0.073905,0.175965,0.0,...,0.0,0.0,0.0,0.0,0.0,0.086808,0.0,0.0,0.033308,0.0
1517,ISIC_0073193,0.0,0.139611,0.0,0.000000,0.043557,0.0,0.073905,0.175965,0.0,...,0.0,0.0,0.0,0.0,0.0,0.086808,0.0,0.0,0.033308,0.0
1518,ISIC_0073195,0.0,0.123884,0.0,0.015693,0.046896,0.0,0.066377,0.167728,0.0,...,0.0,0.0,0.0,0.0,0.0,0.078383,0.0,0.0,0.041129,0.0


48/48 [==============================] - 1s 23ms/step
(1520, 256)


,image,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,...,emb_247,emb_248,emb_249,emb_250,emb_251,emb_252,emb_253,emb_254,emb_255,emb_256
0,ISIC_0000000,0.0,0.139611,0.0,0.000000,0.043557,0.0,0.073905,0.175965,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.086808,0.0,0.000000,0.033308,0.0
1,ISIC_0000001,0.0,0.139611,0.0,0.000000,0.043557,0.0,0.073905,0.175965,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.086808,0.0,0.000000,0.033308,0.0
2,ISIC_0000002,0.0,0.092903,0.0,0.000000,0.052280,0.0,0.085877,0.128976,0.0,...,0.0,0.0,0.032012,0.0,0.0,0.050904,0.0,0.000000,0.068563,0.0
3,ISIC_0000003,0.0,0.139611,0.0,0.000000,0.043557,0.0,0.073905,0.175965,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.086808,0.0,0.000000,0.033308,0.0
4,ISIC_0000004,0.0,0.019288,0.0,0.105425,0.064938,0.0,0.335405,0.161193,0.0,...,0.0,0.0,0.259145,0.0,0.0,0.022893,0.0,0.068641,0.262451,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1515,ISIC_0013235_downsampled,0.0,0.139611,0.0,0.000000,0.043557,0.0,0.073905,0.175965,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.086808,0.0,0.000000,0.033308,0.0
1516,ISIC_0013236_downsampled,0.0,0.105616,0.0,0.004360,0.037977,0.0,0.058930,0.162295,0.0,...,0.0,0.0,0.011361,0.0,0.0,0.069590,0.0,0.000000,0.040387,0.0
1517,ISIC_0013238_downsampled,0.0,0.137349,0.0,0.000000,0.048483,0.0,0.088710,0.174625,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.075484,0.0,0.000000,0.049447,0.0
1518,ISIC_0013242_downsampled,0.0,0.138458,0.0,0.000182,0.045011,0.0,0.076645,0.177364,0.0,...,0.0,0.0,0.001305,0.0,0.0,0.079594,0.0,0.000000,0.032432,0.0


In [24]:
df_complete_seen = df_seen.join(df_embeddings_img_seen.set_index('image'), on='image', how='left')
display(df_complete_seen)

df_complete_test = df_test.join(df_embeddings_img_test.set_index('image'), on='image', how='left')
display(df_complete_test)

,image,age_approx,anatom_site_general,lesion_id,sex,MEL,NV,BCC,AK,BKL,...,emb_247,emb_248,emb_249,emb_250,emb_251,emb_252,emb_253,emb_254,emb_255,emb_256
2915,ISIC_0024318,65.0,lower extremity,HAM_0002450,female,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.086808,0.0,0.0,0.033308,0.0
2967,ISIC_0024370,55.0,unknown,HAM_0001780,male,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.086808,0.0,0.0,0.033308,0.0
2983,ISIC_0024386,40.0,lower extremity,HAM_0005112,female,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.086808,0.0,0.0,0.033308,0.0
3072,ISIC_0024475,35.0,head/neck,HAM_0003873,male,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.086808,0.0,0.0,0.033308,0.0
3114,ISIC_0024517,65.0,posterior torso,HAM_0001894,male,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.086808,0.0,0.0,0.033308,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25256,ISIC_0073141,45.0,anterior torso,BCN_0005520,female,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.086808,0.0,0.0,0.033308,0.0
25259,ISIC_0073144,75.0,anterior torso,BCN_0004091,female,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.086808,0.0,0.0,0.033308,0.0
25287,ISIC_0073193,35.0,anterior torso,BCN_0002147,male,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.086808,0.0,0.0,0.033308,0.0
25289,ISIC_0073195,70.0,lower extremity,BCN_0005492,male,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.078383,0.0,0.0,0.041129,0.0


,image,age_approx,anatom_site_general,lesion_id,sex,MEL,NV,BCC,AK,BKL,...,emb_247,emb_248,emb_249,emb_250,emb_251,emb_252,emb_253,emb_254,emb_255,emb_256
0,ISIC_0000000,55.0,anterior torso,NaN,female,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.086808,0.0,0.000000,0.033308,0.0
1,ISIC_0000001,30.0,anterior torso,NaN,female,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.086808,0.0,0.000000,0.033308,0.0
2,ISIC_0000002,60.0,upper extremity,NaN,female,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.032012,0.0,0.0,0.050904,0.0,0.000000,0.068563,0.0
3,ISIC_0000003,30.0,upper extremity,NaN,male,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.086808,0.0,0.000000,0.033308,0.0
4,ISIC_0000004,80.0,posterior torso,NaN,male,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.259145,0.0,0.0,0.022893,0.0,0.068641,0.262451,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1745,ISIC_0013235_downsampled,45.0,posterior torso,MSK4_0011691,male,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.086808,0.0,0.000000,0.033308,0.0
1746,ISIC_0013236_downsampled,75.0,upper extremity,MSK4_0011692,male,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.011361,0.0,0.0,0.069590,0.0,0.000000,0.040387,0.0
1747,ISIC_0013238_downsampled,75.0,upper extremity,MSK4_0010809,female,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.075484,0.0,0.000000,0.049447,0.0
1748,ISIC_0013242_downsampled,85.0,upper extremity,MSK4_0010725,male,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.001305,0.0,0.0,0.079594,0.0,0.000000,0.032432,0.0


## Train-Val-Test split

In [39]:
columns_cat = ['anatom_site_general', 'sex', 'anatomy_site']
columns_to_drop = [*columns_cat, 'image', 'lesion_id', 'label']

def usable_df(df, columns=None):
    df_dump = df.copy()
    y = df_dump['label']

    for cat in columns_cat:
        one_hot = pd.get_dummies(df_dump[cat], prefix=cat)
        df_dump = df_dump.join(one_hot)

    df_dump = df_dump.drop(columns_to_drop, axis=1)
    if columns is not None:
        missing_cols = set(columns).difference(set(df_dump.columns))
        for miss_col in missing_cols:
            df_dump[miss_col] = np.zeros(df_dump.shape[0])
        df_dump = df_dump[columns]
    return df_dump, df_dump.to_numpy(), y, df_dump.columns

_, X_seen, y_seen, cols_usable = usable_df(df_complete_seen)
X_train, X_val, y_train, y_val = train_test_split(X_seen, y_seen, test_size=TEST_SIZE, random_state=RANDOM_STATE)
print(f"Length of training data: {X_train.shape}")
print(f"Length of validation data: {X_val.shape}")

_, X_test, y_test, _ = usable_df(df_complete_test, cols_usable)
print(f"Length of test data: {X_test.shape}")

Length of training data: (1216, 283)
Length of validation data: (304, 283)
Length of test data: (1520, 283)


## Logistic Regression

In [40]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
from sklearn.linear_model import LogisticRegression

for solver in ["newton-cg", "lbfgs", "liblinear", "sag", "saga"]:
    classifier_logistic = LogisticRegression(random_state=RANDOM_STATE, solver='sag')
    classifier_logistic.fit(X_train, y_train)
    print(f"Train Accuracy (solver={solver}): {classifier_logistic.score(X_train, y_train) * 100:.2f}")
    print(f"Validation Accuracy (solver={solver}): {classifier_logistic.score(X_val, y_val) * 100:.2f}")
    print(f"Test Accuracy (solver={solver}): {classifier_logistic.score(X_test, y_test) * 100:.2f}")

/home/sudha/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Train Accuracy (solver=newton-cg): 100.00
Validation Accuracy (solver=newton-cg): 100.00
Test Accuracy (solver=newton-cg): 100.00


/home/sudha/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Train Accuracy (solver=lbfgs): 100.00
Validation Accuracy (solver=lbfgs): 100.00
Test Accuracy (solver=lbfgs): 100.00


/home/sudha/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Train Accuracy (solver=liblinear): 100.00
Validation Accuracy (solver=liblinear): 100.00
Test Accuracy (solver=liblinear): 100.00


/home/sudha/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Train Accuracy (solver=sag): 100.00
Validation Accuracy (solver=sag): 100.00
Test Accuracy (solver=sag): 100.00
Train Accuracy (solver=saga): 100.00
Validation Accuracy (solver=saga): 100.00
Test Accuracy (solver=saga): 100.00


/home/sudha/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


## Naive Bayes

In [42]:
# https://scikit-learn.org/stable/modules/naive_bayes.html

from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB, BernoulliNB, CategoricalNB
for naive_bayes in [GaussianNB, MultinomialNB, ComplementNB, BernoulliNB, CategoricalNB]:
    classifier_nb = naive_bayes()
    classifier_nb.fit(X_train, y_train)
    print(f"Train Accuracy ({naive_bayes.__name__}): {classifier_nb.score(X_train, y_train) * 100:.2f}")
    print(f"Validation Accuracy ({naive_bayes.__name__}): {classifier_nb.score(X_val, y_val) * 100:.2f}")
    print(f"Test Accuracy ({naive_bayes.__name__}): {classifier_nb.score(X_test, y_test) * 100:.2f}")

Train Accuracy (GaussianNB): 100.00
Validation Accuracy (GaussianNB): 100.00
Test Accuracy (GaussianNB): 99.93
Train Accuracy (MultinomialNB): 99.34
Validation Accuracy (MultinomialNB): 99.34
Test Accuracy (MultinomialNB): 91.84
Train Accuracy (ComplementNB): 100.00
Validation Accuracy (ComplementNB): 100.00
Test Accuracy (ComplementNB): 99.93
Train Accuracy (BernoulliNB): 86.02
Validation Accuracy (BernoulliNB): 82.57
Test Accuracy (BernoulliNB): 71.78
Train Accuracy (CategoricalNB): 100.00
Validation Accuracy (CategoricalNB): 100.00


IndexError: index 1 is out of bounds for axis 1 with size 1

## KNN Classifier

In [43]:
from sklearn.neighbors import KNeighborsClassifier
classifier_knn = KNeighborsClassifier()
classifier_knn.fit(X_train, y_train)
print(f"Train Accuracy: {classifier_knn.score(X_train, y_train) * 100:.2f}")
print(f"Validation Accuracy: {classifier_knn.score(X_val, y_val) * 100:.2f}")
print(f"Test Accuracy: {classifier_knn.score(X_test, y_test) * 100:.2f}")

Train Accuracy: 86.92
Validation Accuracy: 65.13
Test Accuracy: 66.78


## Random Forest

In [44]:
from sklearn.ensemble import RandomForestClassifier
classifier_forest = RandomForestClassifier(max_depth=32, random_state=RANDOM_STATE)
classifier_forest.fit(X_train, y_train)
print(f"Train Accuracy: {classifier_forest.score(X_train, y_train) * 100:.2f}")
print(f"Validation Accuracy: {classifier_forest.score(X_val, y_val) * 100:.2f}")
print(f"Test Accuracy: {classifier_forest.score(X_test, y_test) * 100:.2f}")

Train Accuracy: 100.00
Validation Accuracy: 99.34
Test Accuracy: 82.17


## Adaboost Trees

In [45]:
from sklearn.ensemble import AdaBoostClassifier
classifier_adaboost = AdaBoostClassifier(random_state=RANDOM_STATE)
classifier_adaboost.fit(X_train, y_train)
print(f"Train Accuracy: {classifier_adaboost.score(X_train, y_train) * 100:.2f}")
print(f"Validation Accuracy: {classifier_adaboost.score(X_val, y_val) * 100:.2f}")
print(f"Test Accuracy: {classifier_adaboost.score(X_test, y_test) * 100:.2f}")

Train Accuracy: 75.74
Validation Accuracy: 72.04
Test Accuracy: 25.66


## SVM

In [46]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

classifier_svm = make_pipeline(StandardScaler(), SVC(gamma='auto'))
classifier_svm.fit(X_train, y_train)
print(f"Train Accuracy: {classifier_svm.score(X_train, y_train) * 100:.2f}")
print(f"Validation Accuracy: {classifier_svm.score(X_val, y_val) * 100:.2f}")
print(f"Test Accuracy: {classifier_svm.score(X_test, y_test) * 100:.2f}")

Train Accuracy: 99.92
Validation Accuracy: 95.39
Test Accuracy: 79.67


## RNN

## LSTM

## GRUs